# Modelling NIQS Hardware pyTorch

In [2]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm


from src_torch import *

torch.set_printoptions(precision=8)

## Test

In [2]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
torch.manual_seed(42)

X_target, _, _ = generate_ginibre(d**2, 2)

choi_target = generate_choi(X_target)

state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


X, A, B = generate_ginibre(d**2, 2, requires_grad=True)
optimizer = optim.Adam([A, B], lr=0.01)

fid_list = []

In [ ]:
for i in tqdm(range(10000)):
    optimizer.zero_grad()
    X = A + 1j*B
    choi_model = generate_choi(X)
    index = np.random.randint(0, len(state_input_list)-1)
    state_input = state_input_list[index]
    state_target = state_target_list[index]
    
    state_model = apply_map(state_input, choi_model)
    fid = np.abs(state_fidelity(state_model, state_target).detach().numpy())
    loss = torch.norm(state_model - state_target)
    loss.backward()
    optimizer.step()
    fid_list.append(fid)
    fidelity = state_fidelity
    print(f"step: {i}, fid: {fid:.4f}, norm: {loss.detach().numpy():.4f}")

### Kraus Form

In [4]:
n = 3
d = 2**n

np.random.seed(42)
X, _, _ = generate_ginibre(d, d)
U = generate_unitary(X)

kraus_target = KrausMap(U, c = 0.7, d = d, rank = 2)
choi_target = kraus_to_choi(kraus_target)

num_samples = 1000
input_list = []
target_list = []
for i in range(num_samples):
    config = np.random.randint(6, size=n)
    state = prepare_input(config)
    config = np.random.randint(4, size=n)
    observable = pauli_observable(config)
    
    E = expectation_value(state, observable, kraus_target)
    
    input_list.append([state, observable])
    target_list.append(E)

kraus_model = KrausMap(U, c = 0.1, d = d, rank = 2, requires_grad=True)

model = ModelQuantumMap(model = kraus_model,
                        loss = expectation_value_loss,
                        input_list = input_list,
                        target_list = target_list,
                        lr = 0.05)

model.train(num_iter = 1000)

0: loss: 0.005-0.000j
1: loss: 0.014+0.000j
2: loss: 0.000-0.000j
3: loss: 0.004-0.000j
4: loss: 0.012-0.000j
5: loss: 0.021+0.000j
6: loss: 0.052+0.000j
7: loss: 0.006+0.000j
8: loss: 0.008-0.000j
9: loss: 0.065+0.000j
10: loss: 0.024+0.000j
11: loss: 0.001-0.000j
12: loss: 0.000+0.000j
13: loss: 0.066-0.000j
14: loss: 0.002+0.000j
15: loss: 0.003+0.000j
16: loss: 0.029-0.000j
17: loss: 0.007+0.000j
18: loss: 0.000-0.000j
19: loss: 0.012-0.000j
20: loss: 0.013-0.000j
21: loss: 0.000+0.000j
22: loss: 0.042+0.000j
23: loss: 0.002-0.000j
24: loss: 0.042+0.000j
25: loss: 0.001-0.000j
26: loss: 0.084+0.000j
27: loss: 0.001-0.000j
28: loss: 0.000+0.000j
29: loss: 0.035+0.000j
30: loss: 0.015-0.000j
31: loss: 0.013+0.000j
32: loss: 0.163-0.000j
33: loss: 0.002-0.000j
34: loss: 0.036+0.000j
35: loss: 0.051-0.000j
36: loss: 0.130-0.000j
37: loss: 0.021+0.000j
38: loss: 0.002-0.000j
39: loss: 0.081+0.000j
40: loss: 0.003+0.000j
41: loss: 0.070+0.000j
42: loss: 0.004-0.000j
43: loss: 0.000+0.000

367: loss: 0.015+0.000j
368: loss: 0.017+0.000j
369: loss: 0.000+0.000j
370: loss: 0.003-0.000j
371: loss: 0.015-0.000j
372: loss: 0.025-0.000j
373: loss: 0.003+0.000j
374: loss: 0.018+0.000j
375: loss: 0.023+0.000j
376: loss: 0.004+0.000j
377: loss: 0.081+0.000j
378: loss: 0.000-0.000j
379: loss: 0.030+0.000j
380: loss: 0.013-0.000j
381: loss: 0.029+0.000j
382: loss: 0.074-0.000j
383: loss: 0.002-0.000j
384: loss: 0.002+0.000j
385: loss: 0.000+0.000j
386: loss: 0.000+0.000j
387: loss: 0.027+0.000j
388: loss: 0.001-0.000j
389: loss: 0.008+0.000j
390: loss: 0.000+0.000j
391: loss: 0.009+0.000j
392: loss: 0.003+0.000j
393: loss: 0.024+0.000j
394: loss: 0.013-0.000j
395: loss: 0.000-0.000j
396: loss: 0.004-0.000j
397: loss: 0.023-0.000j
398: loss: 0.124+0.000j
399: loss: 0.014+0.000j
400: loss: 0.184-0.000j
401: loss: 0.001-0.000j
402: loss: 0.058-0.000j
403: loss: 0.033-0.000j
404: loss: 0.159-0.000j
405: loss: 0.033+0.000j
406: loss: 0.006+0.000j
407: loss: 0.078-0.000j
408: loss: 0.028

729: loss: 0.015-0.000j
730: loss: 0.001-0.000j
731: loss: 0.015-0.000j
732: loss: 0.008+0.000j
733: loss: 0.026+0.000j
734: loss: 0.004+0.000j
735: loss: 0.009+0.000j
736: loss: 0.035-0.000j
737: loss: 0.021+0.000j
738: loss: 0.015-0.000j
739: loss: 0.085-0.000j
740: loss: 0.028+0.000j
741: loss: 0.009+0.000j
742: loss: 0.003-0.000j
743: loss: 0.000-0.000j
744: loss: 0.011-0.000j
745: loss: 0.001+0.000j
746: loss: 0.039-0.000j
747: loss: 0.000+0.000j
748: loss: 0.025-0.000j
749: loss: 0.014+0.000j
750: loss: 0.005-0.000j
751: loss: 0.005-0.000j
752: loss: 0.001-0.000j
753: loss: 0.006-0.000j
754: loss: 0.005+0.000j
755: loss: 0.004+0.000j
756: loss: 0.077+0.000j
757: loss: 0.004-0.000j
758: loss: 0.029-0.000j
759: loss: 0.007+0.000j
760: loss: 0.008-0.000j
761: loss: 0.041+0.000j
762: loss: 0.025-0.000j
763: loss: 0.063-0.000j
764: loss: 0.008+0.000j
765: loss: 0.019+0.000j
766: loss: 0.019-0.000j
767: loss: 0.009+0.000j
768: loss: 0.078-0.000j
769: loss: 0.026-0.000j
770: loss: 0.036